In [6]:
import requests
import pickle
import pandas as pd
import time
import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

- 구현 필요 기능
1. API 연결 및 테스트
2. 요청 응답처리
3. API 연결 초기화
4. 통계 코드 검색기능

- 기능으로 구분
- 코드 검색 함수
- Client
- test?

In [59]:
(7+5+5+2+4+1+4+6)/56

0.6071428571428571

In [27]:
('adsf' in 'a')

False

In [43]:
class stats_codes:
    def __init__(self):
        self.stats_codes_info = None
        self.stats_codes = None
    
    def load_stats_code(self, path):
        with open(path,"rb") as f:
            self.stats_codes_info = pickle.load(f)
        self.stats_codes_info = pd.DataFrame(self.stats_codes_info)

    def update_stats_code(self, api_key):
        self.crawling_stats_code()
        stats_list = []
        idx_no = 0
        for stats_code in tqdm(self.stats_codes, desc="Processing stats codes"):
            # url 지정
            url = 'https://ecos.bok.or.kr/api/StatisticItemList/{}/json/kr/1/10/{}'.format(api_key,stats_code)
            
            # API 호출
            response = requests.get(url)
            
            # 응답 데이터 확인
            if response.status_code == 200:
                data = response.json()  # JSON 데이터를 Python 딕셔너리로 변환
            else:
                print(f"Error: {response.status_code}, {response.text}")
            
            idx_no += 1
            if idx_no % 100 == 0:
                print(idx_no, len(stats_list))
                time.sleep(60)

            try : stats_list = stats_list + data['StatisticItemList']['row']
            except : pass

        self.stats_codes_info = pd.DataFrame(stats_list)

    def search_stats_code(self, name):
        if self.stats_codes_info is None :
            print('Empty stats codes info')
        else :
            return self.stats_codes_info.loc[self.stats_codes_info['STAT_NAME'].str.contains(name), :]

    def crawling_stats_code(self):
        # 크롬 드라이버 설정
        chrome_options = Options()
        #chrome_options.add_argument("--headless")  # 브라우저 창을 열지 않고 실행
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # 드라이버 초기화
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        try:
            # 웹 페이지 열기
            url = "https://ecos.bok.or.kr/api/#/DevGuide/StatisticalCodeSearch"  # 크롤링할 URL
            driver.get(url)
            # 스크롤 후 대기
            time.sleep(3)
            # 스크롤 대상 div.rg-scrolltrack 찾기
            scroll_down = driver.find_element(By.CLASS_NAME, "rg-scroll-down")
            # 이미 찾은 mark 태그를 저장할 집합
            found_marks = set()
            # 스크롤하면서 mark 태그 찾기
            while True:
                pre_marks_num = len(found_marks) 
                for i in range(10):
                    scroll_down.click()
                # 스크롤 후 대기
                time.sleep(0.2)
                # mark 태그 찾기
                marks = driver.find_elements(By.CSS_SELECTOR, "mark")
                for mark in marks:
                    found_marks.add(mark.text)
                # 스크롤이 더 이상 움직이지 않을 경우 종료
                if pre_marks_num == len(found_marks) :
                    break
            
            self.stats_codes = [i.split('[')[1].split(']')[0] for i in list(found_marks)]

        finally:
            pass

        print('통계 코드 총 {} 개 확인'.format(len(self.found_marks)))

#### Stats 클래스 기능 테스트

In [44]:
ecos_stat_code = stats_codes()

In [45]:
ecos_stat_code.load_stats_code('stats_df.pkl')

In [46]:
ecos_stat_code.stats_codes_info

,STAT_CODE,STAT_NAME,GRP_CODE,GRP_NAME,ITEM_CODE,ITEM_NAME,P_ITEM_CODE,P_ITEM_NAME,CYCLE,START_TIME,END_TIME,DATA_CNT,UNIT_NAME,WEIGHT
0,901Y024,8.7.6. 농가가계수지,Group1,계정항목,I64A,소득,None,None,A,2003,2023,21,천원,None
1,901Y024,8.7.6. 농가가계수지,Group1,계정항목,I64AA,농가순소득,I64A,소득,A,2003,2023,21,천원,None
2,901Y024,8.7.6. 농가가계수지,Group1,계정항목,I64AAA,농업소득,I64AA,농가순소득,A,2003,2023,21,천원,None
3,901Y024,8.7.6. 농가가계수지,Group1,계정항목,I64AAB,농외소득,I64AA,농가순소득,A,2003,2023,21,천원,None
4,901Y024,8.7.6. 농가가계수지,Group1,계정항목,I64AB,이전소득,I64A,소득,A,2003,2023,21,천원,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5664,271Y012,"2.2.2.2.5. 투입산출표 부가가치유발계수표(생산자가격, 대분류)",Group1,수요부문(열),C04,석탄 및 석유제품,None,None,A,2015,2019,170,None,None
5665,271Y012,"2.2.2.2.5. 투입산출표 부가가치유발계수표(생산자가격, 대분류)",Group1,수요부문(열),C05,화학제품,None,None,A,2015,2019,170,None,None
5666,271Y012,"2.2.2.2.5. 투입산출표 부가가치유발계수표(생산자가격, 대분류)",Group1,수요부문(열),C06,비금속광물제품,None,None,A,2015,2019,170,None,None
5667,271Y012,"2.2.2.2.5. 투입산출표 부가가치유발계수표(생산자가격, 대분류)",Group1,수요부문(열),C07,1차 금속제품,None,None,A,2015,2019,170,None,None


In [50]:
ecos_stat_code.search_stats_code('주식')

,STAT_CODE,STAT_NAME,GRP_CODE,GRP_NAME,ITEM_CODE,ITEM_NAME,P_ITEM_CODE,P_ITEM_NAME,CYCLE,START_TIME,END_TIME,DATA_CNT,UNIT_NAME,WEIGHT
3142,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1010000,KOSPI_회사수,1000000,KOSPI 계열,A,1984,2023,40,사,None
3143,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1010000,KOSPI_회사수,1000000,KOSPI 계열,M,200501,202411,239,사,None
3144,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1020000,KOSPI_종목수,1000000,KOSPI 계열,A,1984,2023,40,종목,None
3145,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1020000,KOSPI_종목수,1000000,KOSPI 계열,M,200501,202411,239,종목,None
3146,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1030000,KOSPI_상장주식수,1000000,KOSPI 계열,A,1984,2023,40,주,None
3147,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1030000,KOSPI_상장주식수,1000000,KOSPI 계열,M,200501,202411,239,주,None
3148,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1040000,KOSPI_시가총액,1000000,KOSPI 계열,A,1984,2023,40,천원,None
3149,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1040000,KOSPI_시가총액,1000000,KOSPI 계열,M,200501,202411,239,천원,None
3150,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1050000,KOSPI_거래량,1000000,KOSPI 계열,A,1984,2023,40,주,None
3151,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1050000,KOSPI_거래량,1000000,KOSPI 계열,M,200501,202411,239,주,None


In [64]:
class api_client:
    def __init__(self, api_key):
        self.api_key = api_key
        self.output_type = 'json'
        self.language = 'kr'
        self.stats_codes = stats_codes()

    def check_api_key(self):
        try :
            sample_url = 'https://ecos.bok.or.kr/api/StatisticTableList/{}/xml/kr/1/10/102Y004'.format(self.api_key)
            response = requests.get(sample_url)
            print('valid API_key : {} '.format(self.api_key))
        except :
            print('API_key not valid : {} '.format(self.api_key))

    def set_output_type(self, output_type):
        if output_type not in ['xml', 'json']:
            print('output type should be "xml" or "json"')
            raise ValueError
        else :
            self.output_type = output_type
            print('set output type : {}'.format(self.output_type))
    
    def set_language(self, language):
        if language not in ['kr', 'en']:
            print('language should be "kr" or "en"')
            raise ValueError
        else :
            self.language = language
            print('language type : {}'.format(self.language))

    def stat_search(self, stat_code, first, end, interval, starttime, endtime, subcode1, subcode2='?', subcode3='?', subcode4='?'):
        url = f'https://ecos.bok.or.kr/api/StatisticSearch/{self.api_key}/{self.output_type}/{self.language}/{first}/{end}/{stat_code}/{interval}/{starttime}/{endtime}/{subcode1}/{subcode2}/{subcode3}/{subcode4}'
        response = requests.get(url)
        # 응답 데이터 확인
        if response.status_code == 200:
            data = response.json()  # JSON 데이터를 Python 딕셔너리로 변환
            self.source_stats_df = pd.DataFrame(data['StatisticSearch']['row'])
            return self.source_stats_df
        else:
            print(f"Error: {response.status_code}, {response.text}")

    def todays_100_stat(self):
        url = f'https://ecos.bok.or.kr/api/KeyStatisticList/{self.api_key}/{self.output_type}/{self.language}/1/101'
        response = requests.get(url)
        # 응답 데이터 확인
        if response.status_code == 200:
            data = response.json()  # JSON 데이터를 Python 딕셔너리로 변환
            self.source_stats_df = pd.DataFrame(data['StatisticSearch']['row'])
            return self.source_stats_df
        else:
            print(f"Error: {response.status_code}, {response.text}")


#### Client 테스트

In [65]:
api_key = 'HLOXWJTZRBR3KCAG5JU3'
client = api_client(api_key)

In [66]:
client.check_api_key()

valid API_key : HLOXWJTZRBR3KCAG5JU3 


In [67]:
client.set_output_type('xls')

output type should be "xml" or "json"


ValueError: 

In [68]:
client.set_output_type('json')

set output type : json


In [69]:
client.set_language('kr')

language type : kr


#### KOSPI 지수 불러오기

- index_no 5310

In [71]:
client.stats_codes.load_stats_code(path='stats_df.pkl')

In [75]:
client.stats_codes.search_stats_code('주식시장')

,STAT_CODE,STAT_NAME,GRP_CODE,GRP_NAME,ITEM_CODE,ITEM_NAME,P_ITEM_CODE,P_ITEM_NAME,CYCLE,START_TIME,END_TIME,DATA_CNT,UNIT_NAME,WEIGHT
3142,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1010000,KOSPI_회사수,1000000,KOSPI 계열,A,1984,2023,40,사,None
3143,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1010000,KOSPI_회사수,1000000,KOSPI 계열,M,200501,202411,239,사,None
3144,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1020000,KOSPI_종목수,1000000,KOSPI 계열,A,1984,2023,40,종목,None
3145,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1020000,KOSPI_종목수,1000000,KOSPI 계열,M,200501,202411,239,종목,None
3146,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1030000,KOSPI_상장주식수,1000000,KOSPI 계열,A,1984,2023,40,주,None
3147,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1030000,KOSPI_상장주식수,1000000,KOSPI 계열,M,200501,202411,239,주,None
3148,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1040000,KOSPI_시가총액,1000000,KOSPI 계열,A,1984,2023,40,천원,None
3149,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1040000,KOSPI_시가총액,1000000,KOSPI 계열,M,200501,202411,239,천원,None
3150,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1050000,KOSPI_거래량,1000000,KOSPI 계열,A,1984,2023,40,주,None
3151,901Y014,"1.5.1.2. 주식시장(월,년)",Group1,계정항목,1050000,KOSPI_거래량,1000000,KOSPI 계열,M,200501,202411,239,주,None


In [70]:
ecos_stat_code.stats_codes_info.loc[5310]

STAT_CODE               802Y001
STAT_NAME      1.5.1.1. 주식시장(일)
GRP_CODE                 Group1
GRP_NAME                   계정항목
ITEM_CODE               0001000
ITEM_NAME               KOSPI지수
P_ITEM_CODE                None
P_ITEM_NAME                None
CYCLE                         D
START_TIME             19950103
END_TIME               20250106
DATA_CNT                   7604
UNIT_NAME       1980.01.04=100 
WEIGHT                     None
Name: 5310, dtype: object

In [63]:
client.stat_search(stat_code='802Y001', first=1, end=7604, interval='D', starttime='19950103', endtime='20250106', subcode1='0001000')

,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,ITEM_CODE2,ITEM_NAME2,ITEM_CODE3,ITEM_NAME3,ITEM_CODE4,ITEM_NAME4,UNIT_NAME,WGT,TIME,DATA_VALUE
0,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,19950103,1013.57
1,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,19950104,997.01
2,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,19950105,976.06
3,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,19950106,975.32
4,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,19950107,988.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7599,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,20241227,2404.77
7600,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,20241230,2399.49
7601,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,20250102,2398.94
7602,802Y001,1.5.1.1. 주식시장(일),0001000,KOSPI지수,None,None,None,None,None,None,1980.01.04=100,None,20250103,2441.92
